In [1]:
#import training data

In [2]:
import json
f = open('train_data.json','r')
train = json.load(f) 
f.close() 

print(len(train["labels"]))

10000


In [10]:
import pandas
df=pandas.read_json("train_data.json",orient="index")
df=df.transpose()
print(df)
df['cat'][10]

                                                 titles labels          cat
0     ImportError: DLL load failed: A dynamic link l...      5     docs-bug
1                                           -1073741795      5          bug
2     zero gradient for higher-order derivatives whe...      1      feature
3     keras lstm layer produces different results in...      1      support
4     Untraced function warning and Model parsing fa...      1  performance
...                                                 ...    ...          ...
9995  TensorFlow/XLA reports an internal error "Dupl...      1         None
9996  Where is the legacy_seq2seq of the tensorflow ...      3         None
9997    misalignment in BahdanauAttention documentation      0         None
9998  Analysis of target '//tensorflow/tools/pip_pac...      5         None
9999  Graph_transformations/propagate_fixed_sizes.cc...      1         None

[10000 rows x 3 columns]


In [3]:
#transform training data word count
from sklearn.feature_extraction.text import CountVectorizer
count_vect=CountVectorizer()
X_train_tf=count_vect.fit_transform(train["titles"])
print(X_train_tf.shape)

(10000, 8327)


In [4]:
#apply inverse document frequency - reduced the weight of the words not necessary in categorizing data
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf=tfidf_transformer.fit_transform(X_train_tf)
print(X_train_tfidf.shape)

(10000, 8327)


In [5]:
#apply naive bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, train["labels"])

In [6]:
#import data to be predict
import json

f = open('tensorflow_opened_issues.json','r')

issues_list = json.load(f) 

print(len(issues_list))

f.close() 

4030


In [7]:
#collect the issues titles, body and labels
data=[]
labels_list=[]
for i in range(len(issues_list)):
    single_data=[]
    issue=issues_list[i]
    issue_title=issue["title"]
    issue_body=issue["body"]
    
    single_data=single_data+[issue_title,issue_body];
    
    data=data+[single_data];
    
    issue_labels_list = issue["labels"]
    single_label_list=[]
    for j in range(len(issue_labels_list)):
        label=issue_labels_list[j]
        label_name=label["name"]
        single_label_list=single_label_list+[label_name]
    labels_list=labels_list+[single_label_list]

In [8]:
print(len(data),len(labels_list))

4030 4030


In [9]:
# collect all the titles of issues
issues_titles=[]
for i in range(len(data)):
    issues_titles=issues_titles+[data[i][0]]
print(len(issues_titles))

4030


In [10]:
#variables to find how many times we are predicting each category
real_cat0=0
real_cat1=0
real_cat2=0
real_cat3=0
real_cat4=0
real_cat5=0
pred_cat0=0
pred_cat1=0
pred_cat2=0
pred_cat3=0
pred_cat4=0
pred_cat5=0
def my_func(a):
    global pred_cat0,pred_cat1,pred_cat2,pred_cat3,pred_cat4,pred_cat5
    if(a==0):
        pred_cat0=pred_cat0+1
    elif(a==1):
        pred_cat1=pred_cat1+1
    elif(a==2):
        pred_cat2=pred_cat2+1
    elif(a==3):
        pred_cat3=pred_cat3+1
    elif(a==4):
        pred_cat4=pred_cat4+1
    elif(a==5):
        pred_cat5=pred_cat5+1



In [11]:
#separate all the issues by their labels or predict labels for them
import re

issue_type=[]
issue_num=[]

predicted_issues=0
non_predicted_issues=0

for i in range(len(labels_list)):
    lb=labels_list[i]
    l=len(lb)
    if(l>0):
        itr=0
        for j in range(len(lb)):
            if(re.search("docs-bug", lb[j])):
                issue_type=issue_type+[['docs-bug']]
                issue_num=issue_num+[0]
                real_cat0=real_cat0+1
                non_predicted_issues=non_predicted_issues+1
                break
            elif(re.search("bug", lb[j])):
                issue_type=issue_type+[['bug']]
                issue_num=issue_num+[1]
                real_cat1=real_cat1+1
                non_predicted_issues=non_predicted_issues+1
                break
            elif(re.search("feature", lb[j])):
                issue_type=issue_type+[['feature']]
                issue_num=issue_num+[2]
                real_cat2=real_cat2+1
                non_predicted_issues=non_predicted_issues+1
                break
            elif(re.search("support", lb[j])):
                issue_type=issue_type+[['support']]
                issue_num=issue_num+[3]
                real_cat3=real_cat3+1
                non_predicted_issues=non_predicted_issues+1
                break
            elif(re.search("performance", lb[j])):
                issue_type=issue_type+[['performance']]
                issue_num=issue_num+[4]
                real_cat4=real_cat4+1
                non_predicted_issues=non_predicted_issues+1
                break
            elif(re.search("build/install", lb[j])):
                issue_type=issue_type+[['build/install']]
                issue_num=issue_num+[5]
                real_cat5=real_cat5+1
                non_predicted_issues=non_predicted_issues+1
                break
            itr=itr+1
        if(itr==l):
            #prediction of unrecognized issue
            issue_type=issue_type+[['unrecognized']] #keep it as unrecognized
            
            #transform test data
            X_test_tf=count_vect.transform([issues_titles[i]])
            X_test_tfidf=tfidf_transformer.transform(X_test_tf)
            predicted = clf.predict(X_test_tfidf)
            
            issue_num=issue_num+[predicted[0]]
            predicted_issues=predicted_issues+1
            my_func(predicted[0])
    else:
        #prediction of unrecognized issue
        issue_type=issue_type+[['unrecognized']] #keep it as unrecognized
        
        #transform test data
        X_test_tf=count_vect.transform([issues_titles[i]])
        X_test_tfidf=tfidf_transformer.transform(X_test_tf)
        predicted = clf.predict(X_test_tfidf)
        
        issue_num=issue_num+[predicted[0]]
        predicted_issues=predicted_issues+1
        my_func(predicted[0])
print(len(issue_type))
print(len(issue_num))

4030
4030


In [12]:
#problem in couning
print("no of total issues: ",len(issue_type))
print("no of predicted issues: ",predicted_issues)
print("no of non-predicted issues: ",non_predicted_issues)

no of total issues:  4030
no of predicted issues:  712
no of non-predicted issues:  3318


In [13]:
#maybe its right
print("no of predicted issues:     ",pred_cat0,pred_cat1,pred_cat2,pred_cat3,pred_cat4,pred_cat5)
print("no of non-predicted issues: ",real_cat0,real_cat1,real_cat2,real_cat3,real_cat4,real_cat5)
print("no of predicted issues:     ",pred_cat0+pred_cat1+pred_cat2+pred_cat3+pred_cat4+pred_cat5)
print("no of non-predicted issues: ",real_cat0+real_cat1+real_cat2+real_cat3+real_cat4+real_cat5)

no of predicted issues:      1 480 0 159 0 72
no of non-predicted issues:  173 1387 773 455 158 372
no of predicted issues:      712
no of non-predicted issues:  3318


In [14]:
#issues and their type
for i in range(len(issue_type)):
    print(issue_type[i],issue_num[i])

['build/install'] 5
['unrecognized'] 1
['performance'] 4
['bug'] 1
['build/install'] 5
['unrecognized'] 1
['unrecognized'] 1
['support'] 3
['docs-bug'] 0
['feature'] 2
['bug'] 1
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 1
['bug'] 1
['build/install'] 5
['build/install'] 5
['unrecognized'] 1
['bug'] 1
['bug'] 1
['unrecognized'] 1
['bug'] 1
['bug'] 1
['unrecognized'] 5
['unrecognized'] 1
['bug'] 1
['bug'] 1
['unrecognized'] 1
['unrecognized'] 1
['performance'] 4
['support'] 3
['support'] 3
['support'] 3
['bug'] 1
['docs-bug'] 0
['unrecognized'] 3
['support'] 3
['support'] 3
['bug'] 1
['support'] 3
['build/install'] 5
['support'] 3
['bug'] 1
['bug'] 1
['bug'] 1
['unrecognized'] 1
['support'] 3
['unrecognized'] 3
['bug'] 1
['bug'] 1
['unrecognized'] 1
['unrecognized'] 1
['support'] 3
['support'] 3
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 5
['performance'] 4
['unrecognized'] 1
['support'] 3
['unrecognized'] 1
['support'] 3
['support'] 3
['unrecognized'] 1
['bug'] 1

['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['build/install'] 5
['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['feature'] 2
['unrecognized'] 1
['feature'] 2
['build/install'] 5
['feature'] 2
['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['feature'] 2
['support'] 3
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['support'] 3
['performance'] 4
['bug'] 1
['bug'] 1
['bug'] 1
['support'] 3
['bug'] 1
['unrecognized'] 1
['build/install'] 5
['bug'] 1
['bug'] 1
['feature'] 2
['support'] 3
['bug'] 1
['docs-bug'] 0
['support'] 3
['feature'] 2
['bug'] 1
['build/install'] 5
['bug'] 1
['bug'] 1
['bug'] 1
['build/install'] 5
['bug'] 1
['feature'] 2
['bug'] 1
['feature'] 2
['performance'] 4
['build/install'] 5
['bug'] 1
['bug'] 1
['support'] 3
['unrecognized'] 1
['feature'] 2
['unrecognized'] 3
['bug'] 1
['bug'] 1
['support'] 3
['unrecognized'] 1
['bug'] 1
['bug'] 1
['build/install'] 5
['support'] 3
['bug'] 1
['bug'] 1
['bug'] 1
['build/install'] 5
['build/install'] 5

['bug'] 1
['bug'] 1
['bug'] 1
['performance'] 4
['bug'] 1
['feature'] 2
['build/install'] 5
['build/install'] 5
['feature'] 2
['bug'] 1
['bug'] 1
['build/install'] 5
['unrecognized'] 1
['bug'] 1
['bug'] 1
['performance'] 4
['bug'] 1
['feature'] 2
['performance'] 4
['feature'] 2
['bug'] 1
['bug'] 1
['feature'] 2
['build/install'] 5
['bug'] 1
['feature'] 2
['bug'] 1
['docs-bug'] 0
['support'] 3
['bug'] 1
['bug'] 1
['unrecognized'] 1
['bug'] 1
['feature'] 2
['unrecognized'] 1
['bug'] 1
['performance'] 4
['bug'] 1
['bug'] 1
['support'] 3
['support'] 3
['bug'] 1
['bug'] 1
['feature'] 2
['bug'] 1
['bug'] 1
['performance'] 4
['bug'] 1
['feature'] 2
['bug'] 1
['feature'] 2
['bug'] 1
['bug'] 1
['support'] 3
['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['feature'] 2
['bug'] 1
['performance'] 4
['unrecognized'] 3
['feature'] 2
['build/install'] 5
['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['feature'] 2
['build/install'] 5
['bug'] 1
['performance'] 4
['bug'] 1
['feature'] 2
['bug'] 1
['support'] 3
['perfo

['bug'] 1
['bug'] 1
['build/install'] 5
['build/install'] 5
['build/install'] 5
['bug'] 1
['bug'] 1
['bug'] 1
['feature'] 2
['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['support'] 3
['unrecognized'] 1
['feature'] 2
['feature'] 2
['bug'] 1
['performance'] 4
['build/install'] 5
['docs-bug'] 0
['bug'] 1
['support'] 3
['bug'] 1
['feature'] 2
['performance'] 4
['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['feature'] 2
['bug'] 1
['support'] 3
['build/install'] 5
['build/install'] 5
['feature'] 2
['performance'] 4
['bug'] 1
['build/install'] 5
['feature'] 2
['unrecognized'] 3
['performance'] 4
['docs-bug'] 0
['bug'] 1
['bug'] 1
['build/install'] 5
['support'] 3
['docs-bug'] 0
['bug'] 1
['bug'] 1
['feature'] 2
['bug'] 1
['support'] 3
['feature'] 2
['bug'] 1
['feature'] 2
['support'] 3
['bug'] 1
['build/install'] 5
['build/install'] 5
['support'] 3
['support'] 3
['bug'] 1
['bug'] 1
['bug'] 1
['unrecognized'] 1
['feature'] 2
['bug'] 1
['bug'] 1
['build/install'] 5
['feature'] 2
['feature'] 2
['build/inst

['bug'] 1
['docs-bug'] 0
['feature'] 2
['feature'] 2
['bug'] 1
['feature'] 2
['bug'] 1
['unrecognized'] 1
['feature'] 2
['support'] 3
['bug'] 1
['feature'] 2
['bug'] 1
['bug'] 1
['feature'] 2
['feature'] 2
['feature'] 2
['feature'] 2
['feature'] 2
['bug'] 1
['bug'] 1
['docs-bug'] 0
['bug'] 1
['feature'] 2
['support'] 3
['build/install'] 5
['bug'] 1
['unrecognized'] 3
['build/install'] 5
['feature'] 2
['feature'] 2
['feature'] 2
['feature'] 2
['bug'] 1
['feature'] 2
['bug'] 1
['build/install'] 5
['support'] 3
['feature'] 2
['bug'] 1
['feature'] 2
['feature'] 2
['build/install'] 5
['bug'] 1
['support'] 3
['bug'] 1
['bug'] 1
['feature'] 2
['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['bug'] 1
['feature'] 2
['unrecognized'] 1
['bug'] 1
['support'] 3
['bug'] 1
['bug'] 1
['bug'] 1
['feature'] 2
['feature'] 2
['bug'] 1
['feature'] 2
['docs-bug'] 0
['bug'] 1
['feature'] 2
['feature'] 2
['docs-bug'] 0
['unrecognized'] 1
['bug'] 1
['feature'] 2
['unrecognized'] 3
['bug'] 1
['bug'] 1
['bug'] 1
['bug']

['feature'] 2
['unrecognized'] 3
['feature'] 2
['unrecognized'] 1
['unrecognized'] 3
['feature'] 2
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 3
['feature'] 2
['unrecognized'] 3
['build/install'] 5
['unrecognized'] 0
['feature'] 2
['unrecognized'] 1
['support'] 3
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 3
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 1
['feature'] 2
['support'] 3
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 3
['feature'] 2
['unrecognized'] 1
['feature'] 2
['feature'] 2
['support'] 3
['feature'] 2
['unrecognized'] 1
['unrecognized'] 5
['unrecognized'] 1
['feature'] 2
['unrecognized'] 1
['support'] 3
['unrecognized'] 3
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 1
['bug'] 1
['bug'] 1
['unrecognized'] 1
['feature'] 2
['unrecognized'] 1
['unrecognized'] 3
['unrecognized'] 1
['bug'] 1
['unrecognized'] 1
['unrecognized'] 1
['unrecognized'] 3
['un

In [15]:
# total number of issues in each category
list1=issue_num
cat0=0
cat1=0
cat2=0
cat3=0
cat4=0
cat5=0
for i in range(len(list1)):
    if(list1[i]==0):
        cat0=cat0+1
    elif(list1[i]==1):
        cat1=cat1+1
    elif(list1[i]==2):
        cat2=cat2+1
    elif(list1[i]==3):
        cat3=cat3+1
    elif(list1[i]==4):
        cat4=cat4+1
    elif(list1[i]==5):
        cat5=cat5+1
print(cat0,cat1,cat2,cat3,cat4,cat5)

174 1867 773 614 158 444
